We will use "IMDB movie review sentiment classification dataset"

Dataset Description: https://keras.io/api/datasets/imdb/

This is a dataset of 25,000 movie reviews from IMDB, tagged by sentiment (positive/negative). The reviews have been preprocessed and each review is coded as a list of (whole) word indexes. For convenience, words are indexed by their overall frequency in the dataset, so that, for example, the integer "3" encodes the 3rd most frequent word in the data.

In [ ]:
!pip install keras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install tensorflow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [11]:
import numpy
import keras
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM, Dropout
from tensorflow.python.keras.layers.embeddings import Embedding
from tensorflow.python.keras.layers.convolutional import Conv1D
from tensorflow.python.keras.layers.convolutional import MaxPooling1D
from tensorflow.keras.preprocessing import sequence
from keras.preprocessing.text import one_hot
from keras.utils import pad_sequences
from tensorflow.python.keras.layers import Flatten
# fix random seed for reproducibility;pl
numpy.random.seed(7)

In [12]:
db=imdb.load_data()

17464789/17464789 [==============================] - 2s 0us/step


In [13]:
top_words = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)

In [14]:
len(X_train)

25000

In [ ]:
y_train

array([1, 0, 0, ..., 0, 1, 0])

In [15]:
max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

In [16]:
X_train.shape

(25000, 500)

we will use the embedding layer which defines the first hidden layer of the network. it must specify 3 arguments:

input_dim: the size of the vocabulary in the text

output_dim: this is the size of the vector space in which each word will be immersed

input_legth: this is the size of the sequence, for example if your documents contain 100 words each then it is 100

In [17]:
# creating tyhe model
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_review_length))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=6, batch_size=64)
print(model.summary())

Epoch 1/6
391/391 [==============================] - 72s 152ms/step - loss: 0.4211 - accuracy: 0.7924 - val_loss: 0.2963 - val_accuracy: 0.8773
Epoch 2/6
391/391 [==============================] - 27s 68ms/step - loss: 0.2400 - accuracy: 0.9070 - val_loss: 0.2980 - val_accuracy: 0.8745
Epoch 3/6
391/391 [==============================] - 17s 45ms/step - loss: 0.1973 - accuracy: 0.9252 - val_loss: 0.2858 - val_accuracy: 0.8833
Epoch 4/6
391/391 [==============================] - 14s 36ms/step - loss: 0.1614 - accuracy: 0.9409 - val_loss: 0.3211 - val_accuracy: 0.8827
Epoch 5/6
391/391 [==============================] - 11s 27ms/step - loss: 0.1459 - accuracy: 0.9486 - val_loss: 0.3117 - val_accuracy: 0.8766
Epoch 6/6
391/391 [==============================] - 12s 32ms/step - loss: 0.1110 - accuracy: 0.9628 - val_loss: 0.3319 - val_accuracy: 0.8758
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape            

In [18]:

# evaluation
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 87.58%


## a simple example of the embedding layer

In [ ]:
docs = ['Well done!',
		'Good work',
		'Great effort',
		'nice work',
		'Excellent!',
		'Weak',
		'Poor effort!',
		'not good',
		'poor work',
		'Could have done better.']

In [ ]:
labels = [1,1,1,1,1,0,0,0,0,0]

In [ ]:
vocab_size = 50

In [ ]:
encoded_docs = [one_hot(d, vocab_size) for d in docs]

In [ ]:
print(encoded_docs)

[[45, 47], [21, 41], [30, 20], [38, 41], [34], [43], [28, 20], [38, 21], [28, 41], [46, 12, 47, 7]]


In [ ]:
max_length = 4
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
print(padded_docs)

[[45 47  0  0]
 [21 41  0  0]
 [30 20  0  0]
 [38 41  0  0]
 [34  0  0  0]
 [43  0  0  0]
 [28 20  0  0]
 [38 21  0  0]
 [28 41  0  0]
 [46 12 47  7]]


We are now ready to define our Embedding layer as part of our model.

The embedding has a vocabulary of 50 and an entry length of 4. We will choose a small embedding space of 8 dimensions.

The model is a simple binary classification model. It is important to note that the output of the Embedding layer will be 4 vectors of 8 dimensions each, one for each word. We flatten it (the flatten layer) into a 32-element vector to pass it to the Dense output layer.

In [ ]:
model = Sequential()
model.add(Embedding(vocab_size, 8, input_length=max_length))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
# summarize the model


In [ ]:
import numpy as np
padded_docs_list = padded_docs.tolist()
model.fit(padded_docs_list, labels, epochs=50, verbose=0)
print(model.summary())

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 module_wrapper_15 (ModuleWr  (None, 4, 8)             400       
 apper)                                                          
                                                                 
 module_wrapper_16 (ModuleWr  (None, 32)               0         
 apper)                                                          
                                                                 
 dense_7 (Dense)             (None, 1)                 33        
                                                                 
Total params: 433
Trainable params: 433
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
loss, accuracy = model.evaluate(padded_docs_list, labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 100.000000


## To Do:

1. Try the same thing on Google reviews dataset ( the file is given in the lab directory)
2. try to change the embedding representation using Glove and Skipgram

In [20]:
import pandas as pd
import numpy as np

# Read the CSV file into a DataFrame
df = pd.read_csv('reviews.csv')
# Display the DataFrame
df.head()


,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,sortOrder,appId
0,Andrew Thomas,https://lh3.googleusercontent.com/a-/AOh14GiHd...,Update: After getting a response from the deve...,1,21,4.17.0.3,2020-04-05 22:25:57,"According to our TOS, and the term you have ag...",2020-04-05 15:10:24,most_relevant,com.anydo
1,Craig Haines,https://lh3.googleusercontent.com/-hoe0kwSJgPQ...,Used it for a fair amount of time without any ...,1,11,4.17.0.3,2020-04-04 13:40:01,It sounds like you logged in with a different ...,2020-04-05 15:11:35,most_relevant,com.anydo
2,steven adkins,https://lh3.googleusercontent.com/a-/AOh14GiXw...,Your app sucks now!!!!! Used to be good but no...,1,17,4.17.0.3,2020-04-01 16:18:13,This sounds odd! We are not aware of any issue...,2020-04-02 16:05:56,most_relevant,com.anydo
3,Lars Panzerbjørn,https://lh3.googleusercontent.com/a-/AOh14Gg-h...,"It seems OK, but very basic. Recurring tasks n...",1,192,4.17.0.2,2020-03-12 08:17:34,We do offer this option as part of the Advance...,2020-03-15 06:20:13,most_relevant,com.anydo
4,Scott Prewitt,https://lh3.googleusercontent.com/-K-X1-YsVd6U...,Absolutely worthless. This app runs a prohibit...,1,42,4.17.0.2,2020-03-14 17:41:01,We're sorry you feel this way! 90% of the app ...,2020-03-15 23:45:51,most_relevant,com.anydo


In [21]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Download the required NLTK resources
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

# Preprocessing functions
def preprocess_text(text):
    # Tokenize the text
    tokens = word_tokenize(text.lower())

    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [token for token in tokens if token not in stop_words]

    # Lemmatize the tokens
    lemmatizer = WordNetLemmatizer()
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]

    # Join the tokens back into a single string
    preprocessed_text = ' '.join(lemmatized_tokens)

    return preprocessed_text

# Apply preprocessing to the 'content' column in the DataFrame
df['preprocessed_content'] = df['content'].apply(preprocess_text)

# Display the preprocessed data

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [22]:
from sklearn.model_selection import train_test_split

reviews = df['preprocessed_content']
score = df['score']
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(reviews, score, test_size=0.2, random_state=42)


In [23]:
vocab_size = 5000
top_words=8000
encoded_revs_train = [one_hot(d, vocab_size) for d in X_train]
encoded_revs_test = [one_hot(d, vocab_size) for d in X_test]

In [24]:
from sklearn.preprocessing import OneHotEncoder

# define one hot encoding
encoder = OneHotEncoder(sparse=False)
y_train = np.array(y_train).reshape(-1,1)
y_test = np.array(y_test).reshape(-1,1)
# transform data
y_train = encoder.fit_transform(y_train)
y_test = encoder.fit_transform(y_test)
print(y_train)

[[0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 ...
 [0. 0. 1. 0. 0.]
 [0. 0. 0. 1. 0.]
 [1. 0. 0. 0. 0.]]


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [25]:
max_length_train = 500 #393
X_train = sequence.pad_sequences(encoded_revs_train, maxlen=max_length_train)
X_test =  sequence.pad_sequences(encoded_revs_test, maxlen=max_length_train)

In [26]:
# creating tyhe model
embedding_vecor_length = 100
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length=max_length_train))
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=64, kernel_size=3, padding='same', activation='relu'))
model.add(LSTM(200))
model.add(Dense(5, activation='sigmoid'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=7, batch_size=64)
print(model.summary())

Epoch 1/7
197/197 [==============================] - 37s 165ms/step - loss: 1.3960 - accuracy: 0.3944 - val_loss: 1.2205 - val_accuracy: 0.4746
Epoch 2/7
197/197 [==============================] - 14s 68ms/step - loss: 1.0714 - accuracy: 0.5511 - val_loss: 1.1414 - val_accuracy: 0.5289
Epoch 3/7
197/197 [==============================] - 9s 44ms/step - loss: 0.8572 - accuracy: 0.6516 - val_loss: 1.1125 - val_accuracy: 0.5797
Epoch 4/7
197/197 [==============================] - 8s 39ms/step - loss: 0.6358 - accuracy: 0.7511 - val_loss: 1.1536 - val_accuracy: 0.6105
Epoch 5/7
197/197 [==============================] - 7s 35ms/step - loss: 0.4810 - accuracy: 0.8169 - val_loss: 1.1491 - val_accuracy: 0.6219
Epoch 6/7
197/197 [==============================] - 6s 32ms/step - loss: 0.3533 - accuracy: 0.8711 - val_loss: 1.3214 - val_accuracy: 0.6524
Epoch 7/7
197/197 [==============================] - 6s 29ms/step - loss: 0.2690 - accuracy: 0.9033 - val_loss: 1.3374 - val_accuracy: 0.6695
Mod

### 2. try to change the embedding representation using Glove and Skipgram :

In [1]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

--2023-06-18 14:15:27--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2023-06-18 14:15:28--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2023-06-18 14:15:28--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [27]:
embedding_dim = 100  # or 300, depending on the GloVe version you downloaded
import numpy as np
embeddings_index = {}
with open('glove.6B.100d.txt', encoding='utf-8') as f:  # or 'glove.6B.300d.txt'
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs


In [28]:
#from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
num_words = len(tokenizer.word_index) + 1
embedding_matrix = np.zeros((num_words, embedding_dim))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector


In [37]:
max_review_length = 500
model = Sequential()
model.add(Embedding(num_words, embedding_dim, weights=[embedding_matrix], input_length=max_review_length, trainable=False))


In [38]:
model.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(LSTM(100))
model.add(Dense(5, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=6, batch_size=64)

scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1] * 100))


Epoch 1/6
197/197 [==============================] - 6s 16ms/step - loss: 1.5712 - accuracy: 0.3215 - val_loss: 1.5736 - val_accuracy: 0.3108
Epoch 2/6
197/197 [==============================] - 3s 16ms/step - loss: 1.5680 - accuracy: 0.3226 - val_loss: 1.5746 - val_accuracy: 0.3108
Epoch 3/6
197/197 [==============================] - 3s 14ms/step - loss: 1.5683 - accuracy: 0.3226 - val_loss: 1.5739 - val_accuracy: 0.3108
Epoch 4/6
197/197 [==============================] - 3s 13ms/step - loss: 1.5674 - accuracy: 0.3226 - val_loss: 1.5729 - val_accuracy: 0.3108
Epoch 5/6
197/197 [==============================] - 3s 13ms/step - loss: 1.5674 - accuracy: 0.3226 - val_loss: 1.5739 - val_accuracy: 0.3108
Epoch 6/6
197/197 [==============================] - 5s 24ms/step - loss: 1.5676 - accuracy: 0.3226 - val_loss: 1.5732 - val_accuracy: 0.3108
Accuracy: 31.08%
